In [ ]:
from platform import python_version
print(python_version())

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import argparse
import logging
from pathlib import Path

import numpy as np

In [ ]:
import sys
sys.path.append('../../source/')


In [ ]:

import HolographicLattices

In [ ]:
from HolographicLattices.DifferentialOperators.DifferentialOperator import *
from HolographicLattices.Equations.EquationsOfMotion import EquationsOfMotion
from HolographicLattices.Equations.Jacobian import FiniteDifferenceJacobian
from HolographicLattices.IO.SeedLoader import load_seed_algebraic
from HolographicLattices.Physics.Observables import Observables
from HolographicLattices.LinearSolver.SimpleLinearSolver import SimpleLinearSolver
from HolographicLattices.NonLinearSolver.NRNonLinearSolver import NewtonRaphsonFiniteDifferenceBVPSolver as NRSolver
from HolographicLattices.Options import Options
from HolographicLattices.Utilities import GridUtilities

In [ ]:
    # Basic argparse for some settings. Can be extended, but not required.
    parser = argparse.ArgumentParser()

    parser.add_argument("--setupfile", type=Path, help="Setup .yaml file to read from")
    parser.add_argument("--output", type=Path, nargs="?", help="Where to output")
    parser.add_argument("--log", type=str, default="INFO",
                        help="Logging level to print, choice of [NONE|DEBUG|INFO|WARN|ERROR|CRITICAL]")
    parser.add_argument("--logfile", type=Path, default="SolveWithFD.log", help="Location of log file")
    parser.add_argument("--seedfile", type=Path, nargs="?")

    parsed, rest = parser.parse_known_args()

    logging.basicConfig(format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", filename=parsed.logfile,
                        level=parsed.log)


In [ ]:
    # Load all the options
    #setupfile = parsed.setupfile
    
    setupfile="1DLattice_setup.yaml"

    io_options       = Options.IOOptions.load_from_file(setupfile)
    equation_options = Options.EquationOptions.load_from_file(setupfile)
#    grid_options     = Options.GridOptions.load_from_file(setupfile)
    solver_options   = Options.NonLinearSolverOptions.load_from_file(setupfile)
    constant_options = Options.ConstantOptions.load_from_file_safe(setupfile)

    grid_dimensions = equation_options.grid_sizes

In [ ]:
equations_of_motion = EquationsOfMotion(constant_options=constant_options, equation_options=equation_options)

equations_of_motion.load_from_folder(io_options.coefficient_folder, equation_options.grid_periodic)

grid_information = GridUtilities.GridInformation(equation_options)

In [ ]:
differential_operator = MixedFDDSpectralDifferentialOperator.construct_differentiation_matrices(
            equation_options=equation_options, max_deriv=2
        )

In [ ]:
np.ravel

In [ ]:
linear_solver = SimpleLinearSolver()

In [ ]:
jacobian = FiniteDifferenceJacobian(
            constant_options=constant_options,
            equation_options=equation_options,
            finite_difference_matrices=differential_operator.get_matrix_representation()
        )

In [ ]:
jacobian.load_from_folder(folder=io_options.coefficient_folder, periodicities=equation_options.grid_periodic)


In [ ]:
observables = Observables.parse_observables(io_options.observables_file, constant_options, equation_options)

In [ ]:
nonlinear_solver = NRSolver(solver_options=solver_options, differential_operator=differential_operator,
                                equations_of_motion=equations_of_motion, linear_solver=linear_solver, jacobian=jacobian,
                                grid_information=grid_information, observables=observables)

In [ ]:
seed = load_seed_algebraic(io_options.seed_file, constant_options, grid_information)

In [ ]:
seed.shape

In [ ]:
_, _, success = nonlinear_solver.solve(seed)

In [ ]:
if success:
    result = nonlinear_solver.last_result
    result.write_to_file(io_options.output_file)

In [ ]:
import HolographicLattices.FiniteDifferences.FiniteDifferenceMatrices

In [ ]:
HolographicLattices.Utilities.GridUtilities.get_all_derivatives(2, 2)

In [ ]:
equation_options.max_deriv

In [ ]:
seed = load_seed_algebraic(io_options.seed_file, constant_options, grid_info)

In [ ]:
seed.shape